In [20]:
import sys
print(sys.executable)



c:\Users\adhik\OneDrive\Desktop\job_recommendation_system\venv\Scripts\python.exe


In [21]:
import pandas as pd
import numpy as np
import os
import pickle


In [22]:
df = pd.read_csv("../backend/models/jobs_with_features.csv")
print("Loaded:", df.shape)
df.head(3)


Loaded: (986, 18)


,jobid,job_title,description_text,benefits,job_type,location,salary_formatted,company_name,company_rating,company_reviews_count,country,url,apply_link,job_title_lc,company_name_lc,location_lc,combined_text,description_short
0,d46480488e3aa20c,FitLab Email Marketing Manager,Description: \n Who Is FitLab: FitLab is a p...,"[""401(k)"",""Dental insurance"",""Disability insur...",Full-time,California,"$90,000 - $105,000 a year",FitLab INC,4.5,4,US,https://www.indeed.com/viewjob?jk=d46480488e3a...,https://www.indeed.com/viewjob?jk=d46480488e3a...,fitlab email marketing manager,fitlab inc,california,fitlab email marketing manager description who...,Description: \n Who Is FitLab: FitLab is a p...
1,ba9bf4e04cfd9cdb,Customer Service Representative - Work From Home,The Customer Associate is the first point of c...,"[""401(k) matching"",""Continuing education credi...",Permanent,"Sugar Land, TX 77479",$28 - $50 an hour,Buccal Up Dental,5.0,4,US,https://www.indeed.com/viewjob?jk=ba9bf4e04cfd...,https://www.indeed.com/viewjob?jk=ba9bf4e04cfd...,customer service representative - work from home,buccal up dental,"sugar land, tx 77479",customer service representative work from home...,The Customer Associate is the first point of c...
2,9e4031d06dd42d32,Information Technology Support Specialist,About Peraton Peraton is a next-generation na...,"[""Dental insurance"",""Disability insurance"",""He...",Full-time,United States,"$66,000 - $106,000 a year",Peraton,3.1,297,US,https://www.indeed.com/viewjob?jk=9e4031d06dd4...,https://www.indeed.com/viewjob?jk=9e4031d06dd4...,information technology support specialist,peraton,united states,information technology support specialist abou...,About Peraton Peraton is a next-generation na...


In [23]:
assert "combined_text" in df.columns, "combined_text column missing. Run Notebook 02 again."


In [24]:
print("Empty combined_text:", (df["combined_text"].fillna("").str.len() == 0).sum())
print("Unique job titles:", df["job_title"].nunique() if "job_title" in df.columns else "missing")
print("Unique companies:", df["company_name"].nunique() if "company_name" in df.columns else "missing")


Empty combined_text: 0
Unique job titles: 947
Unique companies: 512


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    stop_words="english",
    max_features=7000,     # 5000–10000 is fine; 7000 is a good middle
    ngram_range=(1, 2)     # unigrams + bigrams improves matching
)


In [26]:
X_tfidf = tfidf.fit_transform(df["combined_text"])
print("TF-IDF matrix shape:", X_tfidf.shape)


TF-IDF matrix shape: (986, 7000)


In [27]:
os.makedirs("../backend/models", exist_ok=True)

with open("../backend/models/tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(tfidf, f)

# Save sparse matrix
from scipy.sparse import save_npz
save_npz("../backend/models/tfidf_matrix.npz", X_tfidf)

print("Saved tfidf_vectorizer.pkl and tfidf_matrix.npz")


Saved tfidf_vectorizer.pkl and tfidf_matrix.npz


In [28]:
from scipy.sparse import load_npz

X_loaded = load_npz("../backend/models/tfidf_matrix.npz")
print("Loaded TF-IDF matrix shape:", X_loaded.shape)


Loaded TF-IDF matrix shape: (986, 7000)


In [29]:
from sklearn.metrics.pairwise import cosine_similarity

def recommend_tfidf(
    profile_text: str,
    top_n: int = 10,
    location: str | None = None,
    job_type: str | None = None
) -> pd.DataFrame:
    
    profile_text = str(profile_text).lower().strip()
    if not profile_text:
        raise ValueError("profile_text is empty. Enter some skills/keywords.")

    q_vec = tfidf.transform([profile_text])
    sims = cosine_similarity(q_vec, X_tfidf).flatten()

    results = df.copy()
    results["score"] = sims

    # Optional filtering
    if location and location.lower() != "any":
        results = results[results["location"].str.contains(location, case=False, na=False)]

    if job_type and job_type.lower() != "any":
        results = results[results["job_type"].str.contains(job_type, case=False, na=False)]

    results = results.sort_values("score", ascending=False).head(top_n)

    # return minimal columns (safe if some columns missing)
    show_cols = [c for c in [
        "job_title", "company_name", "location", "job_type", "salary_formatted", "company_rating", "score", "url", "apply_link"
    ] if c in results.columns]

    return results[show_cols]


In [30]:
recommend_tfidf("frontend developer react internship", top_n=5)


,job_title,company_name,location,job_type,salary_formatted,company_rating,score,url,apply_link
585,Java Developer Level 2,Kforce,"Cincinnati, OH 45202",Contract,$67 - $72 an hour,3.8,0.336049,https://www.indeed.com/viewjob?jk=c328667a127a...,https://www.indeed.com/viewjob?jk=c328667a127a...
484,"Principal Product Manager, Technical, AWS Deve...","Amazon Web Services, Inc.","San Francisco, CA",Full-time,"$163,000 - $281,800 a year",3.4,0.236713,https://www.indeed.com/viewjob?jk=9da8b5c8a56a...,https://www.indeed.com/viewjob?jk=9da8b5c8a56a...
507,"Senior Product Manager, Technical, AWS Develop...","Amazon Web Services, Inc.","San Francisco, CA",Full-time,"$137,000 - $236,800 a year",3.4,0.225627,https://www.indeed.com/viewjob?jk=f17d8cad571c...,https://www.indeed.com/viewjob?jk=f17d8cad571c...
265,Senior Applications Developer,Buildertrend,"Omaha, NE 68137",Full-time,"$115,000 - $165,000 a year",3.6,0.122074,https://www.indeed.com/viewjob?jk=c269f320bff3...,https://www.indeed.com/viewjob?jk=c269f320bff3...
150,Quantitative Analyst/Developer,Corebridge Financial,"New York, NY",Full-time,"$108,000 - $135,000 a year",2.9,0.108838,https://www.indeed.com/viewjob?jk=7c70a0bf4efa...,https://www.indeed.com/viewjob?jk=7c70a0bf4efa...


In [31]:
recommend_tfidf("data analyst python sql excel", top_n=5)


,job_title,company_name,location,job_type,salary_formatted,company_rating,score,url,apply_link
631,Data Analyst,DAP Health,"Palm Springs, CA 92262",Full-time,$32.49 - $33.97 an hour,3.1,0.275674,https://www.indeed.com/viewjob?jk=0415f5a87724...,https://www.indeed.com/viewjob?jk=0415f5a87724...
807,Data Analyst - Immigration Data,Fragomen,"New York, NY",Full-time,"$69,000 - $78,000 a year",3.3,0.262707,https://www.indeed.com/viewjob?jk=3d5e7c1c3550...,https://www.indeed.com/viewjob?jk=3d5e7c1c3550...
844,Data Analyst,Slate Magazine,"Brooklyn, NY",Full-time,"$100,000 - $115,000 a year",4.0,0.233804,https://www.indeed.com/viewjob?jk=0766f46e90db...,https://www.indeed.com/viewjob?jk=0766f46e90db...
839,HR Data Analyst,Antech Diagnostics,"New York, NY 10013",Full-time,"$85,000 - $110,000 a year",2.9,0.218180,https://www.indeed.com/viewjob?jk=715dc5bb3d26...,https://www.indeed.com/viewjob?jk=715dc5bb3d26...
527,Data Analyst (Remote),Medwatch LLC,Remote,Full-time,"$80,000 - $90,000 a year",3.3,0.194841,https://www.indeed.com/viewjob?jk=5a0e36d66b9f...,https://www.indeed.com/viewjob?jk=5a0e36d66b9f...


In [32]:
recommend_tfidf("customer service remote", top_n=5, location="Remote")


,job_title,company_name,location,job_type,salary_formatted,company_rating,score,url,apply_link
548,Customer Care Associate,Health Advocate,Remote,Full-time,From $16 an hour,2.7,0.370881,https://www.indeed.com/viewjob?jk=d1d56a9b2031...,https://www.indeed.com/viewjob?jk=d1d56a9b2031...
118,Benefits Specialist,Health Advocate,Remote,Full-time,From $19 an hour,2.7,0.338031,https://www.indeed.com/viewjob?jk=a32251ea8a17...,https://www.indeed.com/viewjob?jk=a32251ea8a17...
568,"Customer Service (Irwindale, CA) (Remote in CA...","TDB Communications, Inc.",Remote,Full-time,$18 an hour,3.4,0.264578,https://www.indeed.com/viewjob?jk=e3d672802788...,https://www.indeed.com/viewjob?jk=e3d672802788...
609,Dental Customer Service Representative – Remote,Mutual of Omaha,Remote,Full-time,$18.25 - $19.25 an hour,3.7,0.255236,https://www.indeed.com/viewjob?jk=a39db3bc8a0a...,https://www.indeed.com/viewjob?jk=a39db3bc8a0a...
584,Customer Service Representative,One80 Intermediaries,Remote,Full-time,"$37,400 a year",3.0,0.245323,https://www.indeed.com/viewjob?jk=fa152416bd88...,https://www.indeed.com/viewjob?jk=fa152416bd88...


In [33]:
feature_names = np.array(tfidf.get_feature_names_out())


In [34]:
def explain_match(profile_text: str, row_index: int, top_terms: int = 8):
    q_vec = tfidf.transform([profile_text.lower()])
    job_vec = X_tfidf[row_index]

    overlap = q_vec.multiply(job_vec)

    if overlap.nnz == 0:
        return []

    top_idx = overlap.indices[np.argsort(overlap.data)[::-1][:top_terms]]
    return feature_names[top_idx].tolist()


In [35]:
results = recommend_tfidf("frontend developer react internship", top_n=5)
top_title = results.iloc[0]["job_title"]

# find the index in df for that job title (safe approach)
top_row_index = df.index[df["job_title"] == top_title][0]

explain_match("frontend developer react internship", top_row_index, top_terms=10)


['developer']

In [36]:
def recommend_tfidf_json(profile_text, top_n=10, location=None, job_type=None):
    recs = recommend_tfidf(profile_text, top_n, location, job_type)
    recs = recs.copy()

    # Add explanation terms for each recommended row
    explanations = []
    for _, r in recs.iterrows():
        # locate row index by matching title+company+location (more stable)
        mask = (df["job_title"] == r.get("job_title","")) & (df["company_name"] == r.get("company_name","")) & (df["location"] == r.get("location",""))
        if mask.any():
            idx = df.index[mask][0]
            explanations.append(explain_match(profile_text, idx, 8))
        else:
            explanations.append([])

    recs["why"] = explanations
    return recs.to_dict(orient="records")

recommend_tfidf_json("data analyst python sql", top_n=3)


[{'job_title': 'Data Analyst',
  'company_name': 'DAP Health',
  'location': 'Palm Springs, CA 92262',
  'job_type': 'Full-time',
  'salary_formatted': '$32.49 - $33.97 an hour',
  'company_rating': 3.1,
  'score': 0.268283349257971,
  'url': 'https://www.indeed.com/viewjob?jk=0415f5a877249856',
  'apply_link': 'https://www.indeed.com/viewjob?jk=0415f5a877249856',
  'why': ['data analyst', 'data', 'sql', 'python', 'analyst']},
 {'job_title': 'Data Analyst - Immigration Data',
  'company_name': 'Fragomen',
  'location': 'New York, NY',
  'job_type': 'Full-time',
  'salary_formatted': '$69,000 - $78,000 a year',
  'company_rating': 3.3,
  'score': 0.2581851832708183,
  'url': 'https://www.indeed.com/viewjob?jk=3d5e7c1c3550b28a',
  'apply_link': 'https://www.indeed.com/viewjob?jk=3d5e7c1c3550b28a',
  'why': ['data analyst', 'data', 'analyst']},
 {'job_title': 'Data Analyst',
  'company_name': 'Slate Magazine',
  'location': 'Brooklyn, NY',
  'job_type': 'Full-time',
  'salary_formatted': 